<a href="https://colab.research.google.com/github/faan03/Schnorr/blob/main/Schnorr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gensafeprime
import gensafeprime as gen
import random as random

class Zq:
    
    def __init__(self,q):
        self.q = q        

    def __modulo (self,a,b):
        return a%b # residuo div entera entre a y b

    def suma(self,a,b):
        r= self.__modulo((a+b),self.q)
        return r

    def resta(self,a,b):
        r = self.__modulo (a-b,self.q)
        return r

    def producto (self,a,b):
        r= self.__modulo(a*b,self.q)
        return r 

    def division (self,a,b):
        invM_b= self.invMulti(b)
        r= self.__modulo(a*invM_b,self.q)
        return r

    def potencia (self,base, potencia):   
        r= pow(base,potencia,self.q)
        return r

    def invMulti(self,a):
        r= self.__gcdExtended(a,self.q)[1]
        return r    

    def __gcdExtended(self,a, b): 
    # Base Case 
        if a == 0 :  
            return b,0,1             
        gcd,x1,y1 = self.__gcdExtended(b%a, a) 
        # Update x and y using results of recursive 
        # call 
        x = y1 - (b//a) * x1 
        y = x1      
        return gcd,x,y        

    def getRandomElement(self):
        return random.randint(0,self.q-1)

    def getRandomNotZeroElement(self):
        return random.randint(1,self.q-1)   

class schnorr_G:
    def __init__(self,G,q):
        self.G=G
        self.q=q        
        self.zq = Zq(q)
        self.__getGenerator()       

    def generateKeyPair(self):
        sk= self.zq.getRandomNotZeroElement()       
        pk = self.G.potencia (self.g,sk)
        return (sk,pk)      

    def __getGenerator(self):
        ##toma un elemento entre 1 y p-1        
        element= self.G.getRandomElement()
        while ((self.G.potencia (element,2)==1) or (self.G.potencia(element,self.q)!=1)):
            element= self.G.getRandomElement()
        self.g = element

    def get_g(self):             
        return self.g   

class schnorr_P:
    
    def __init__(self,G,q,g,alfa):        
        self.q=q
        self.G = G
        self.zq = Zq(q)          
        self.g = g
        self.alfa=alfa
    
        
    def computeAlfa_t(self):        
        self.alfa_t = self.zq.getRandomNotZeroElement() #  q 
        u_t = self.G.potencia (self.g,self.alfa_t) # u_t = g^alfa_t 
        return (u_t)    
    
    def computeAlfaZ (self,c):   
        alfaZ= self.zq.suma(self.alfa_t,self.zq.producto(self.alfa,c))
        return alfaZ
        

class schnorr_V:
    def __init__(self,G,q,g,u):        
        self.G = G
        self.zq = Zq(q)        
        self.u =u
        self.g = g    
    

    def set_u_t(self,ut):
        self.ut=ut    

    def getC(self):
        self.c= self.zq.getRandomElement()
        return self.c

    def check (self,alfaZ):
        l= self.G.potencia(self.g,alfaZ)
        r=  self.G.producto(self.ut, self.G.potencia(self.u,self.c))
        if (l==r):
            return True
        else :
            return False    


In [8]:
p = gen.generate(100)
q=((p-1)//2)
G= Zq(p)
Gs = schnorr_G(G,q)
(sk, pk) = Gs.generateKeyPair()

P = schnorr_P(G,q,Gs.get_g(),sk)

V = schnorr_V(G,q,Gs.get_g(),pk)

u_t=P.computeAlfa_t()
V.set_u_t(u_t)
cValue= V.getC()
alfaZ=P.computeAlfaZ(cValue)
print (V.check(alfaZ))



True
